In [ ]:
    base = 'gdrive/MyDrive/vocr/'

#Runtime Info
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')


Thu Dec  8 14:38:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   78C    P0    35W /  70W |    374MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#setup
from google.colab import drive
drive.mount('/content/gdrive')
!rm -rf Partitioned-Organic-Data-Split
print("Unzipping...")
!unzip {base}Partitioned-Organic-Data-Split.zip>/dev/null
print("Done!")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import numpy as np
import random as rn
import tensorflow as tf
import os
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(0)
rn.seed(0)
tf.random.set_seed(0)
from datetime import datetime
from glob import glob
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from random import randint
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_fscore_support
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import Sequential
from tensorflow.keras.applications import MobileNetV3Small, MobileNetV3Large, ResNet50V2
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger
from tensorflow.keras.layers import Dense, add, Dropout, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D, BatchNormalization, Activation, Input, Concatenate, Cropping2D, ActivityRegularization, RandomBrightness, RandomContrast, RandomCrop, RandomFlip, RandomHeight, RandomRotation, RandomTranslation, RandomWidth, RandomZoom
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.utils import Sequence
from tensorflow.keras.utils import to_categorical
import codecs
import cv2
import pandas as pd
import time
from google.colab import runtime
from scipy.io import loadmat
import multiprocessing
tf.keras.mixed_precision.set_global_policy('mixed_float16')
tf.get_logger().setLevel('ERROR')

In [ ]:
# Util functions
def log(message):
	print(message)
	file = codecs.open(base+"log.txt", "a", "cp1252", "replace")
	print(message, file=file)

def load_img(path, label):
	img = tf.io.read_file(path)
	img = tf.io.decode_png(img, channels=3)
	img = tf.cast(img, tf.float16)
	img = tf.image.resize(img, (224,224))
	return img, label


In [ ]:
class MetricCallback(Callback):
	def __init__(self, x, y, k=0, patience=0, restore_best=True):
		super().__init__()
		self.x = x
		self.y = y
		self.k=k
		self.best_score = 0.0
		self.best_epoch = 0
		self.best_weights = None
		self.patience = patience
		self.restore_best = restore_best

	def restore(self):
		self.model.set_weights(self.best_weights)

	def on_epoch_end(self, epoch, logs={}):
		preds = self.model.predict(self.x)
		final = f1_score(self.y.argmax(-1), preds.argmax(-1), average='micro')
		if final>self.best_score:
			for f in glob("*.h5"): os.remove(f)
			self.model.save(f"model {self.k+1} {epoch+1} f{final:.3f}.h5")
			!rm {base}*.h5
			!cp *.h5 {base}
			log(f"{epoch+1} Best Score: improved from {self.best_score:.3f} to {final:.3f}")
			self.best_score = final
			self.best_epoch = epoch
			self.best_weights = self.model.get_weights()
		else: log(f"{epoch+1} Score: {final:.3f}, Best epoch: {self.best_epoch+1}, {self.best_score:.3f}")

		if self.patience>0 and epoch-self.best_epoch >= self.patience:
			print("Stopping...")
			self.stopped_epoch = epoch
			self.model.stop_training = True
			if self.restore_best: self.restore()
			return

		if not epoch == self.best_epoch and (epoch-self.best_epoch)%int(self.patience/2) == 0:
			old_lr = float(tf.keras.backend.get_value(self.model.optimizer.lr))
			new_lr = old_lr * 0.1
			tf.keras.backend.set_value(self.model.optimizer.lr, new_lr)
			self.restore()
			log(f"Reducing learning rate from {old_lr} to {new_lr}")


In [ ]:
# Model
def describe(model):
	for i, layer in enumerate(model.layers):
		if layer.trainable: print(i, layer.name, layer.trainable, layer.input_shape, layer.output_shape)
		if hasattr(layer, "layers"): describe(layer)

def create_model(input_shape, output_shape):
	data_augmentation = Sequential([
		RandomBrightness(0.2),
		RandomContrast(0.2),
		RandomFlip(),
		RandomRotation(0.2),
		RandomTranslation(0.2, 0.2),
		RandomZoom(0.2, 0.2)
	])
	base_model = MobileNetV3Small(include_top=False, input_shape=input_shape, weights='imagenet', pooling="avg")
	base_model.trainable = False
	inputs = Input(shape=input_shape)
	x = data_augmentation(inputs)
	x = base_model(x, training=False)
	x = Dense(256)(x)
	x = BatchNormalization()(x)
	x = Activation("relu")(x)
	x = Dropout(0.2)(x)
	outputs =Dense(output_shape,activation='softmax')(x)
	model=Model(inputs=inputs, outputs=outputs)
	model.compile(optimizer=Adam(1e-3), loss='CategoricalCrossentropy',metrics=['accuracy'])
	describe(model)
	print(model.summary())
	return model


In [ ]:
#Imblearn
def balance(x, y):
	print("Balancing samples...")
	ros = RandomOverSampler()
	x, y = ros.fit_resample(x.reshape(-1,1), y)
	# Can't use synthetic for memory limitation
	#smote_enn = SMOTEENN(random_state=0)
	#x, y = smote_enn.fit_resample(x, y)
	x = x.reshape(-1)
	u, c = np.unique(y, return_counts=True)
	names = [le.classes_[l] for l in u]
	dist = list(zip(names, c))
	dist = sorted(dist, key = lambda x: x[1])
	print("Distribution after over sampling:", dist)
	return x, y
	

In [ ]:
size = (224, 224, 3)
batch_size = 32
k=0
t_start = time.time()
now = datetime.now()
date_time = now.strftime("%m/%d/%Y, %H:%M:%S")
log(f"Starting at: {date_time}")
log(f"Size: {size}, batch_size: {batch_size}, k_fold: {k}")
log(f"Tensorflow: {tf.version.VERSION}")
log(f"SKLearn: {sklearn.__version__}")


Starting at: 12/08/2022, 14:40:23
Size: (224, 224, 3), batch_size: 32, k_fold: 0
Tensorflow: 2.9.2
SKLearn: 1.0.2


In [ ]:
train = loadmat("Partitioned-Organic-Data-Split/organic-training.mat")
x_train = train['trainingImages'].flatten()
x_train = [x[0][x[0].index("Partitioned"):] for x in x_train]
x_train = np.array(x_train)
y_train = train['labels'].flatten().astype(int)
le = LabelEncoder()
y_train = le.fit_transform(y_train)
weight = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
weight = dict(zip(np.unique(y_train), weight))
y_train= to_categorical(y_train)

validation = loadmat("Partitioned-Organic-Data-Split/organic-validation.mat")
x_validation = validation['trainingImages'].flatten()
x_validation = [x[0][x[0].index("Partitioned"):] for x in x_validation]
x_validation = np.array(x_validation)
y_validation = validation['labels'].flatten().astype(int)
y_validation = le.transform(y_validation)
y_validation = to_categorical(y_validation)

test = loadmat("Partitioned-Organic-Data-Split/organic-test.mat")
x_test = test['trainingImages'].flatten()
x_test = [x[0][x[0].index("Partitioned"):] for x in x_test]
x_test = np.array(x_test)
y_test = test['labels'].flatten().astype(int)
y_test = le.transform(y_test)
y_test = to_categorical(y_test)
log(f"x_train: {x_train.shape}, y_train:{y_train.shape}, x_test: {x_test.shape}, y_test: {y_test.shape}")


['TextView', 'ImageView', 'Button', 'Switch', 'EditText', 'ImageButton', 'CheckedTextView', 'ProgressBarHorizontal', 'RatingBar', 'ProgressBarVertical', 'ToggleButton', 'CheckBox', 'Spinner', 'SeekBar', 'NumberPicker', 'RadioButton']
True True


In [ ]:
#Train
log(f"Training fold {k}...")
model = create_model(size, y_train.shape[1])
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds= train_ds.shuffle(x_train.shape[0]).map(load_img, num_parallel_calls=batch_size).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
validation_ds = tf.data.Dataset.from_tensor_slices((x_validation, y_validation))
validation_ds= validation_ds.map(load_img, num_parallel_calls=batch_size).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
mc = MetricCallback(validation_ds, y_validation, patience=50)
logger = CSVLogger(base+'train.csv')
callbacks = [mc, logger]
history = model.fit(train_ds, validation_data=validation_ds, epochs=1000, callbacks =callbacks, workers=multiprocessing.cpu_count(), use_multiprocessing=True)


Training fold 0...
0 input_38 True [(None, 224, 224, 3)] [(None, 224, 224, 3)]
1 sequential_18 True (None, 224, 224, 3) (None, 224, 224, 3)
0 random_brightness_18 True (None, 224, 224, 3) (None, 224, 224, 3)
1 random_contrast_18 True (None, 224, 224, 3) (None, 224, 224, 3)
2 random_flip_18 True (None, 224, 224, 3) (None, 224, 224, 3)
3 random_rotation_18 True (None, 224, 224, 3) (None, 224, 224, 3)
4 random_translation_18 True (None, 224, 224, 3) (None, 224, 224, 3)
5 random_zoom_18 True (None, 224, 224, 3) (None, 224, 224, 3)
3 dense_36 True (None, 576) (None, 256)
4 batch_normalization_18 True (None, 256) (None, 256)
5 activation_18 True (None, 256) (None, 256)
6 dropout_18 True (None, 256) (None, 256)
7 dense_37 True (None, 256) (None, 16)
Model: "model_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_38 (InputLayer)       [(None, 224, 224, 3)]     0         
                               

KeyboardInterrupt: ignored

In [ ]:
#Predict
mc.restore()
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds= test_ds.map(load_img, num_parallel_calls=batch_size).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
preds = model.predict(test_ds)
score = precision_recall_fscore_support(y_test.argmax(-1), preds.argmax(-1), average='micro')
print(f"Micro Precision: {score[0]:.2f}, Recall: {score[1]:.2f}, F1: {score[2]:.2f}")
score = precision_recall_fscore_support(y_test.argmax(-1), preds.argmax(-1), average='weighted')
print(f"Weighted Precision: {score[0]:.2f}, Recall: {score[1]:.2f}, F1: {score[2]:.2f}")
score = precision_recall_fscore_support(y_test.argmax(-1), preds.argmax(-1), average='macro')
print(f"Macro Precision: {score[0]:.2f}, Recall: {score[1]:.2f}, F1: {score[2]:.2f}")
matrix = confusion_matrix(y_test.argmax(-1), preds.argmax(-1))
print("Confusion Matrix:")
for i,m in enumerate(matrix):
  row = [(le.classes_[j], k) for j, k in enumerate(m) if k>0]
  print(i, le.classes_[i], row)


In [ ]:
#End
now = datetime.now()
date_time = now.strftime("%m/%d/%Y, %H:%M:%S")
log(f"Ending at: {date_time}")
t_finish = time.time()
total_time = (t_finish-t_start)/60
log(f"It took {total_time:.2f} minutes")
time.sleep(1)
#runtime.unassign()